In [3]:
import pandas as pd
import numpy as np
from pycountry_convert import country_name_to_country_alpha3
from pycountry_convert import country_alpha3_to_country_alpha2
from pycountry_convert import country_alpha2_to_country_name
from pycountry_convert import map_countries

In [139]:
entry_visas = pd.read_csv('../visa-per-country.csv')[17:]

In [5]:
applications_made = pd.read_csv('../data/applications made.csv')

In [140]:
entry_visas

,Quarter,Geographical region,Country of nationality,Total visas granted,Total visas granted (excl. visitors and transit),Total Work,Total Study (excl. Short-term students)
17,2005 Q1,Asia Central,Afghanistan,877,465,1,8
18,2005 Q1,Europe Other,Albania,"2,460",413,18,27
19,2005 Q1,Africa North,Algeria,"2,391",238,21,38
20,2005 Q1,Oceania,American Samoa,0,0,0,0
21,2005 Q1,Europe Other,Andorra,0,0,0,0
22,2005 Q1,Africa Sub-Saharan,Angola,787,96,4,59
23,2005 Q1,Other,Anguilla (British),0,0,0,0
24,2005 Q1,America Central and South,Antigua and Barbuda,8,7,1,1
25,2005 Q1,America Central and South,Argentina,220,211,98,28
26,2005 Q1,Europe Other,Armenia,301,49,12,21


In [159]:
year_of_interest = '2016'

In [146]:
entry_visas.columns

Index(['Quarter', 'Geographical region', 'Country', 'Total\nvisas granted',
       'Total\nvisas granted\n(excl. visitors\nand transit)', 'Total Work',
       'Total Study\n(excl. Short-term students)'],
      dtype='object')

In [147]:
entry_visas.rename(columns={'Total\nvisas granted': 'Total',
                            'Country of nationality': 'Country'}, inplace=True)

In [172]:
filename = 'map_of_immigration_2005.csv'
filename[:19] + 'work_' + filename[19:]

'map_of_immigration_work_2005.csv'

In [149]:
for year_of_interest in [str(year) for year in np.arange(2005,2017)]:
    entry_visas_2016 = pd.DataFrame(columns=["Year", "Total", "Country", "Geographical region"])

    entry_visas_2016['Year'] = entry_visas['Quarter'].apply(lambda year: year[:4])
    entry_visas_2016['Country'] = entry_visas['Country']
    entry_visas_2016['Geographical region'] = entry_visas['Geographical region']

    entry_visas_2016['Total'] = entry_visas['Total'].apply(lambda number: ('0' if ('z' in number) else number))
    entry_visas_2016['Total'] = entry_visas_2016['Total'].apply(lambda number: number.replace(',',''))
    entry_visas_2016['Total'] = pd.to_numeric(entry_visas_2016['Total'])

    entry_visas_2016 = entry_visas_2016[entry_visas_2016['Year']==year_of_interest][17:]

    entry_visas_2016.Country.replace(to_replace=conversion_of_country_names, inplace=True)

    entry_visas_2016['country_code'] = entry_visas_2016['Country'].apply(country_to_code)
    entry_visas_2016_refugees = entry_visas_2016[entry_visas_2016['country_code']=='Refugees']

    df = entry_visas_2016[entry_visas_2016['country_code'].apply(len) <= 3]
    grouped = df.groupby(by='Country').sum()
    grouped.reset_index(inplace=True)
    grouped['country_code'] = grouped['Country'].apply(country_to_code)


    grouped['Geographical region'] = grouped['Country'].replace(dictionary_of_regions)

    list_of_country_codes = list(set([country_to_code(country) for country in countries]))

    grouped.set_index('country_code',inplace=True)

    for code in list_of_country_codes:
        if code not in grouped.index:
            country_name = country_alpha2_to_country_name(country_alpha3_to_country_alpha2(code))
            grouped.loc[code] = [country_name,0,'Other']

    grouped.reset_index(inplace=True)

    grouped.to_csv('../data/map_of_immigration_all_{}.csv'.format(year_of_interest))

In [132]:
entry_visas_2016['Year'] = entry_visas['Quarter'].apply(lambda year: year[:4])
entry_visas_2016['Country'] = entry_visas['Country']
entry_visas_2016['Geographical region'] = entry_visas['Geographical region']

In [133]:
entry_visas_2016['Total'] = entry_visas['Total Study'].apply(lambda number: ('0' if ('z' in number) else number))

In [134]:
entry_visas_2016['Total'] = entry_visas_2016['Total'].apply(lambda number: number.replace(',',''))

In [135]:
entry_visas_2016['Total'] = pd.to_numeric(entry_visas_2016['Total'])

In [138]:
entry_visas_2016 = entry_visas_2016[entry_visas_2016['Year']=='2016'][17:]

In [20]:
# Converts some of the country names to standard

conversion_of_country_names = {'Korea (North)':'North Korea','Korea (South)':'South Korea',
                               'Occupied Palestinian Territories':'the State of Palestine',
                              'Virgin Islands (British)':'British Virgin Islands',
                              'Virgin Islands (US)':'United States Virgin Islands',
                              'Burma':'Myanmar'}

In [21]:
countries = map_countries().keys()

In [22]:
def country_to_code(country): 
    try:
        code = country_name_to_country_alpha3(country)
    except KeyError:

        # If there is no exact match try sub-strings
        for country_name in countries:
            if country_name in country:
                code = country_name_to_country_alpha3(country_name)
                break
            elif country in country_name:
                code = country_name_to_country_alpha3(country_name)
                break
        else:
            code = country
        
    return code

In [150]:
entry_visas_2016['country_code'] = entry_visas_2016['Country'].apply(country_to_code)

## Separates refugees and deletes very small entries

In [151]:
entry_visas_2016_refugees = entry_visas_2016[entry_visas_2016['country_code']=='Refugees']

df = entry_visas_2016[entry_visas_2016['country_code'].apply(len) <= 3]
grouped = df.groupby(by='Country').sum()
grouped.reset_index(inplace=True)
grouped['country_code'] = grouped['Country'].apply(country_to_code)

# Creates dictionary of geographical regions

In [24]:
dictionary_of_regions = {item['Country']:item['Geographical region'] for [_,item] in df.iterrows()}

In [155]:
# Fills the country codes that do not show up in the table with zeros

In [156]:
list_of_country_codes = list(set([country_to_code(country) for country in countries]))

grouped.set_index('country_code',inplace=True)

for code in list_of_country_codes:
    if code not in grouped.index:
        country_name = country_alpha2_to_country_name(country_alpha3_to_country_alpha2(code))
        grouped.loc[code] = [country_name,0,'Other']
        
grouped.reset_index(inplace=True)

In [14]:
grouped.to_csv('data/map_of_immigration_2017.csv')

In [18]:
grouped2 = pd.read_csv('data/map_of_immigration_2017.csv', index_col=0)

In [19]:
grouped2

,country_code,Country,Total,Geographical region
0,AFG,Afghanistan,21,Asia Central
1,ALB,Albania,75,Europe Other
2,DZA,Algeria,106,Africa North
3,ASM,American Samoa,0,Oceania
4,AND,Andorra,3,Europe Other
5,AGO,Angola,37,Africa Sub-Saharan
6,AIA,Anguilla (British),0,Other
7,ATG,Antigua and Barbuda,2,America Central and South
8,ARG,Argentina,385,America Central and South
9,ARM,Armenia,39,Europe Other


## Plotting a map of immigration

In [13]:
# Experimenting with plotly

import plotly
import plotly.plotly as py

from plotly.graph_objs import *

In [422]:
plotly.tools.set_credentials_file(username='Campello', api_key='zFzFwmvrlVOPaKseNrb0')
plotly.tools.set_config_file(world_readable=True, sharing='public')

In [437]:
data = [ dict(
        type = 'choropleth',
        locations = grouped['country_code'],
        z = grouped['Total'],
        text = grouped['Country'] + '<br>' + grouped['Geographical region'],
        colorscale = [[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
            [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            len=0.7,
            title = 'Total work visas')
      ), dict(
        type = 'choropleth',
        locations = ['GBR'],
        z = [1],
        text = ['United Kingdom'],
        colorscale = [[0,"rgb(255, 10, 17)"],[1,"rgb(255, 10, 17)"]],
        showscale=False)]

layout = dict(
    title = 'Map of legal work immigration<br>Source:\
            <a href="https://www.gov.uk/government/publications/immigration-statistics-october-to-december-2016/work#data-tables">\
            Home Office</a>',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
py.plot(fig, validate=False, filename='d3-world-map' )

'https://plot.ly/~Campello/6'

In [1]:
import pandas as pd

/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
df = pd.read_csv('data/map_of_immigration_2017.csv', index_col=0)

In [151]:
import numpy as np

year_and_total = []
for year in np.arange(2005,2017):
    total = pd.read_csv('../data/map_of_immigration_all_{}.csv'.format(year))['Total'].sum()
    year_and_total += [[year, total]]
    
pd.DataFrame(year_and_total).to_csv('../data/total_year_all.csv', index=0, header=['year','Total'])

In [29]:
sorted_countryes = df.sort_values(by='Total',ascending=False)[:8][['Country','Total']].get_values()

In [41]:
df.index

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            245, 246, 247, 248, 249, 250, 251, 252, 253, 254],
           dtype='int64', length=255)

In [43]:
pd.read_csv('data/total_year.csv', index_col=0)

,Total
year,
2005,243267
2006,249479
2007,205079
2008,184156
2009,155551
2010,160638
2011,149239
2012,145037
2013,154693


In [30]:
sorted_countryes

array([['India', 59180],
       ['Australia', 15336],
       ['United States', 13820],
       ['Philippines', 13119],
       ['Canada', 5801],
       ['New Zealand', 5479],
       ['China', 5339],
       ['Japan', 4920]], dtype=object)

In [26]:
def func(a,b,c):
    d = 1
    print(locals())

In [27]:
func(1,2,3)

{'d': 1, 'c': 3, 'b': 2, 'a': 1}


## Loading applications made

In [137]:
entry_visas = pd.read_csv('../data/applications made.csv')
    
entry_visas.head()

,Geographical region,Country of nationality,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,*Total,*Total,"2,564,252","2,731,079","2,541,865","2,440,369","2,447,391","2,529,962","2,607,142","2,561,623","2,830,012","2,748,348","2,840,390","2,893,466"
1,Africa North,*Total Africa North,"88,757","98,003","89,794","95,096","97,478","104,426","90,494","103,250","124,185","117,877","111,458","115,245"
2,Africa Sub-Saharan,*Total Africa Sub-Saharan,"517,203","466,354","406,022","384,610","433,473","432,010","436,475","430,172","418,843","379,081","377,526","324,554"
3,America North,*Total America North,"44,321","46,250","46,716","50,510","51,917","53,815","49,767","47,843","51,049","50,924","50,349","47,701"
4,America Central and South,*Total America Central and South,"75,061","77,307","74,049","70,161","68,373","67,163","61,900","62,354","66,177","79,664","81,659","74,516"


In [125]:
for year_of_interest in [str(year) for year in np.arange(2005,2017)]:
    print('Loading year {}'.format(year_of_interest), end='\r')
    # Loads applied visa statistics

    entry_visas = pd.read_csv('../data/applications made.csv')
    entry_visas.rename(columns={'Country of nationality': 'Country', year_of_interest: 'Total'}, inplace=True)

    entry_visas_one_year = pd.DataFrame(columns=["Year", "Total", "Country", "Geographical region"])

    entry_visas_one_year['Country'] = entry_visas['Country']
    entry_visas_one_year['Geographical region'] = entry_visas['Geographical region']

    entry_visas_one_year['Total'] = entry_visas['Total'].apply(lambda number: ('0' if ('z' in number) else number))
    entry_visas_one_year['Total'] = entry_visas_one_year['Total'].apply(lambda number: number.replace(',',''))
    entry_visas_one_year['Total'] = pd.to_numeric(entry_visas_one_year['Total'])

    entry_visas_one_year.Country.replace(to_replace=conversion_of_country_names, inplace=True)

    entry_visas_one_year['country_code'] = entry_visas_one_year['Country'].apply(country_to_code)
    entry_visas_one_year_refugees = entry_visas_one_year[entry_visas_one_year['country_code']=='Refugees']

    df = entry_visas_one_year[entry_visas_one_year['country_code'].apply(len) <= 3]
    grouped_applications = df.groupby(by='Country').sum()
    grouped_applications.reset_index(inplace=True)
    grouped_applications['country_code'] = grouped_applications['Country'].apply(country_to_code)

    grouped_applications['Geographical region'] = grouped_applications['Country'].replace(dictionary_of_regions)

    list_of_country_codes = list(set([country_to_code(country) for country in countries]))

    grouped_applications.set_index('country_code',inplace=True)

    dictionary_of_regions = {item['Country']:item['Geographical region'] for [_,item] in df.iterrows()}

    for code in list_of_country_codes:
        if code not in grouped_applications.index:
            country_name = country_alpha2_to_country_name(country_alpha3_to_country_alpha2(code))
            grouped_applications.loc[code] = [country_name,0,'Other']

    grouped_applications.reset_index(inplace=True)


    # Loads granted visa statistics

    entry_visas = pd.read_csv('../visa-per-country.csv')[17:]
    entry_visas.rename(columns={'Total\nvisas granted\n(excl. visitors\nand transit)': 'Total',
                                'Country of nationality': 'Country'}, inplace=True)


    entry_visas_one_year = pd.DataFrame(columns=["Year", "Total", "Country", "Geographical region"])

    entry_visas_one_year['Year'] = entry_visas['Quarter'].apply(lambda year: year[:4])
    entry_visas_one_year['Country'] = entry_visas['Country']
    entry_visas_one_year['Geographical region'] = entry_visas['Geographical region']

    entry_visas_one_year['Total'] = entry_visas['Total'].apply(lambda number: ('0' if ('z' in number) else number))
    entry_visas_one_year['Total'] = entry_visas_one_year['Total'].apply(lambda number: number.replace(',',''))
    entry_visas_one_year['Total'] = pd.to_numeric(entry_visas_one_year['Total'])

    entry_visas_one_year = entry_visas_one_year[entry_visas_one_year['Year']==year_of_interest][17:]

    entry_visas_one_year.Country.replace(to_replace=conversion_of_country_names, inplace=True)

    entry_visas_one_year['country_code'] = entry_visas_one_year['Country'].apply(country_to_code)
    entry_visas_one_year_refugees = entry_visas_one_year[entry_visas_one_year['country_code']=='Refugees']

    df = entry_visas_one_year[entry_visas_one_year['country_code'].apply(len) <= 3]
    grouped = df.groupby(by='Country').sum()
    grouped.reset_index(inplace=True)
    grouped['country_code'] = grouped['Country'].apply(country_to_code)


    grouped['Geographical region'] = grouped['Country'].replace(dictionary_of_regions)

    list_of_country_codes = list(set([country_to_code(country) for country in countries]))

    grouped.set_index('country_code',inplace=True)

    for code in list_of_country_codes:
        if code not in grouped.index:
            country_name = country_alpha2_to_country_name(country_alpha3_to_country_alpha2(code))
            grouped.loc[code] = [country_name,0,'Other']

    grouped.reset_index(inplace=True)

    # Calculates what is left

    grouped_rejections = grouped_applications
    grouped_rejections['Total'] = grouped_applications['Total']-grouped['Total']

    grouped_rejections.to_csv('../data/map_of_immigration_rejected_{}.csv'.format(year_of_interest))

In [128]:
grouped_rejections['Total'].sum()

412760

In [132]:
grouped['Total'].sum()

2466497